## Import packages

In [156]:
%%time
import os
import csv
import requests
import random
import json
import pandas as pd
import numpy as np
import haversine as hs
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

CPU times: user 32 µs, sys: 5 µs, total: 37 µs
Wall time: 42 µs


## Read data

In [18]:
# read json function
def read_json_from_file(file_name):
    with open(file_name, 'r') as fd:
        data = fd.read()
    return json.loads(data)

In [19]:
# read csv function
def read_restaurant_list(file_name):
    with open(file_name, newline='') as f:
        reader = csv.reader(f)
        data = list(reader)
    return data[0]

In [67]:
%%time
# read order data
select_df = pd.read_csv("../Data/select_milkrun_order_data.csv")
select_df = select_df.drop(select_df.columns[0], axis=1)

# read order per time data
order_per_time_period = pd.read_csv("../Data/order_per_time_period_2.csv")
order_per_time_period.columns = ['time_period_id', 'create_time_period', 'number']
order_per_time_period = order_per_time_period.loc[(order_per_time_period['time_period_id'] >=4) & (order_per_time_period['time_period_id'] <= 11)].reset_index(drop=True)
order_per_time_period['ratio'] = order_per_time_period['number'] / sum(order_per_time_period['number'])

# read restaurant data
restaurant_list = read_restaurant_list("../Data/restaurant_list.csv")
restaurant_customer_dict = read_json_from_file("../Data/restaurant_customer_dict.json")
restaurant_coordinate_dict = read_json_from_file("../Data/restaurant_coordinate_dict.json")

# time period (from 9:00AM to 15:00PM)
order_per_time_period_dict = pd.Series(order_per_time_period.number.values, index=order_per_time_period.index).to_dict()
order_percent_per_time_period_dict = pd.Series(order_per_time_period.ratio.values, index=order_per_time_period.index).to_dict()
time_id_period_dict = pd.Series(order_per_time_period.create_time_period.values, index=order_per_time_period.index).to_dict()
time_period_id = list(order_per_time_period_dict.keys())
restaurant_order_ratio = select_df['pick_up_address'].value_counts().to_dict()

CPU times: user 505 ms, sys: 290 ms, total: 795 ms
Wall time: 982 ms


## Order information generation

In [47]:
def order_num_generation(select_restaurants, total_order_number):
    '''
    Generate orders for each restaurant in each time period
    Inputs:
    - select_restaurants: list of select_restaurants
    - total_order_number: total number of orders from all restaurants
    '''
    restaurant_order_number = {r:[] for r in select_restaurants}
    restaurant_order_percent = {r:[] for r in select_restaurants}
    restaurant_order_per_time_period = {r: [] for r in select_restaurants}
    
    number = sum([restaurant_order_ratio[r] for r in select_restaurants])
    for r in select_restaurants:
        restaurant_order_percent[r] = restaurant_order_ratio[r] / number
        restaurant_order_number[r] = round(total_order_number * restaurant_order_percent[r])
        for t in time_period_id:
            order_num = round(restaurant_order_number[r] * order_percent_per_time_period_dict[t])
            restaurant_order_per_time_period[r].append([t, order_num])
    
    return restaurant_order_per_time_period

In [48]:
def order_time_generation(t1, t2, num_orders):
    '''
    Generate random order create time in time interval [t1, t2]
    Inputs:
    - t1: start time
    - t2: end time
    - num_orders: number of orders create in [t1, t2]
    Output:
    - list of order create time
    '''
    num = 0
    output = []
    
    while num < num_orders:
        integer = random.randint(0, (t2 - t1).total_seconds())
        step = timedelta(seconds=integer)
        order_time = t1 + step
        output.append(order_time)
        num = num + 1
    
    output.sort()
    
    return [i.strftime('%Y-%m-%d %H:%M:%S') for i in output]

In [49]:
def order_id_generation(time_list):
    '''
    Input:
    - time_list: list of order create time, output of Function order_time_generation
    Output:
    - list of order id
    '''
    output = []
    replace_char = "-: "
    
    for o in time_list:
        for c in replace_char:
            o = o.replace(c,"")
        output.append(o)
    
    return output

In [50]:
def order_location_generation(order_id_list, restaurant):
    '''
    Input:
    - order_id_list
    - restaurant
    Output:
    - list of orders location
    '''
    o_location_list = random.sample(restaurant_customer_dict[restaurant], len(order_id_list))
    return o_location_list

In [161]:
def instance_generation(restaurants, restaurant_order_num):
    '''
    Inputs:
    - restaurants： select restaurants
    Output:
    - Instance
    '''
    instance_df = pd.DataFrame(columns=['order_id', 'demand', 'creation_time', 'committed_completion_time', 
                                        'load_time', 'unload_time', 'pickup_id', 'delivery_id'])
    
    # one instance (from 6 AM to 22:30 PM)
    for time_id in time_period_id:

        # for each time period
        start = datetime.strptime(time_id_period_dict[time_id], "%Y-%m-%d %H:%M:%S")
        end = datetime.strptime(time_id_period_dict[time_id], "%Y-%m-%d %H:%M:%S") + timedelta(minutes=45)

        # for each restaurant
        for r in restaurants:
            small_df = pd.DataFrame()

            order_time_list = order_time_generation(start, end, num_orders=restaurant_order_num[r][time_id][1])
            order_id_list = order_id_generation(order_time_list)
            order_location_list = order_location_generation(order_id_list, r)
            
            small_df['order_id'] = order_id_list
            small_df['creation_time'] = order_time_list
            small_df['restaurant_name'] = [r] * len(order_id_list)
            small_df['customer_name'] = [o[0] for o in order_location_list]
            small_df['pick_up_coordinate'] = [restaurant_coordinate_dict[r]] * len(order_id_list)
            small_df['delivery_coordinate'] = [o[1] for o in order_location_list]         

            instance_df = instance_df.append(small_df, ignore_index=True)
    
    # update instance dataframe
    instance_df['demand'] = 1
    instance_df['load_time'] = 60
    instance_df['unload_time'] = 60
    instance_df['creation_time'] = pd.to_datetime(instance_df['creation_time'])
    instance_df = instance_df.sort_values(by='creation_time').reset_index()
    instance_df['committed_completion_time'] = instance_df['creation_time'] + timedelta(minutes=45)
    
    # only keep time no date
    instance_df['creation_time'] = instance_df['creation_time'].dt.time
    instance_df['committed_completion_time'] = instance_df['committed_completion_time'].dt.time
    
    # add pickup and delivery location
    instance_df['pickup_id'] = instance_df['restaurant_name']
    instance_df['delivery_id'] = instance_df['customer_name']
    
    instance_df.drop(['index'], axis=1, inplace=True)
    
    for index, row in instance_df.iterrows():
        instance_df.at[index, 'order_id'] = row['order_id'][-6:] + str(index).zfill(5)

    return instance_df

### Generate instance

In [211]:
# restaurant_list = ['Ghim Moh Market & Food Centre',
#                    'Bukit Timah Market & Food Centre',
#                    'Alexandra Village Food Centre']
# restaurant_list = ['Kovan Food Centre (209 Hougang Street 21)',
#                    'Breadtalk Ihq, 30 Tai Seng St, Singapore 534013, Singapore',
#                    'Bedok Interchange Hawker Centre',
#                    'Chomp Chomp',
#                    'Lorong 8 Toa Payoh Hawker Centre',
#                    'Old Airport Road Food Centre']
restaurant_list = ['Alexandra Village Food Centre',
                   'Bukit Merah View Market & Hawker Centre',
                   '417 River Valley Rd, Singapore 248316, Singapore',
                   'Zion Riverside Food Centre',
                   'Tiong Bahru Market',
                   '37 Cuppage Rd, Singapore 229460, Singapore',
                   '371 Beach Rd, Singapore 199597, Singapore',
                   'Golden Mile Food Centre',
                   '66 Kg Bugis, Singapore 338987, Singapore',
                   '36 Boat Quay, Singapore 049825, Singapore',
                   '54 South Bridge Rd, Singapore 058685, Singapore',
                   'Hong Lim Food Centre',
                   'Maxwell Food Centre',
                   'Amoy Street Food Centre',
                   'Old Airport Road Food Centre'
                  ]

1600 order with 100 drivers and 10 mins time interval still havs delays with total 138 seconds, while 5mins no delay.

For 15 restaurants, 1600 orders with 100 drivers has no delay, 80 drivers will have 54329s delay

In [212]:
restaurant_order_number = order_num_generation(restaurant_list, 1600)
restaurant_order_number

{'Alexandra Village Food Centre': [[0, 18],
  [1, 24],
  [2, 26],
  [3, 23],
  [4, 13],
  [5, 11],
  [6, 8],
  [7, 9]],
 'Bukit Merah View Market & Hawker Centre': [[0, 16],
  [1, 21],
  [2, 24],
  [3, 20],
  [4, 12],
  [5, 10],
  [6, 8],
  [7, 8]],
 '417 River Valley Rd, Singapore 248316, Singapore': [[0, 7],
  [1, 10],
  [2, 11],
  [3, 9],
  [4, 5],
  [5, 4],
  [6, 3],
  [7, 4]],
 'Zion Riverside Food Centre': [[0, 8],
  [1, 11],
  [2, 12],
  [3, 10],
  [4, 6],
  [5, 5],
  [6, 4],
  [7, 4]],
 'Tiong Bahru Market': [[0, 17],
  [1, 23],
  [2, 26],
  [3, 22],
  [4, 13],
  [5, 10],
  [6, 8],
  [7, 9]],
 '37 Cuppage Rd, Singapore 229460, Singapore': [[0, 19],
  [1, 25],
  [2, 28],
  [3, 24],
  [4, 14],
  [5, 11],
  [6, 9],
  [7, 9]],
 '371 Beach Rd, Singapore 199597, Singapore': [[0, 5],
  [1, 6],
  [2, 7],
  [3, 6],
  [4, 4],
  [5, 3],
  [6, 2],
  [7, 2]],
 'Golden Mile Food Centre': [[0, 13],
  [1, 17],
  [2, 19],
  [3, 16],
  [4, 10],
  [5, 8],
  [6, 6],
  [7, 7]],
 '66 Kg Bugis, Singa

In [213]:
instance = instance_generation(restaurants=restaurant_list, restaurant_order_num=restaurant_order_number)
instance

,order_id,demand,creation_time,committed_completion_time,load_time,unload_time,pickup_id,delivery_id,restaurant_name,customer_name,pick_up_coordinate,delivery_coordinate
0,09003000000,1,09:00:30,09:45:30,60,60,Hong Lim Food Centre,161126 Singapore,Hong Lim Food Centre,161126 Singapore,"[1.28532379322674, 103.845776735598]","[1.282856548, 103.8294964]"
1,09003100001,1,09:00:31,09:45:31,60,60,Tiong Bahru Market,165006 Singapore,Tiong Bahru Market,165006 Singapore,"[1.28508829297302, 103.832348811484]","[1.286545996, 103.8308245]"
2,09003500002,1,09:00:35,09:45:35,60,60,Hong Lim Food Centre,164027 Singapore,Hong Lim Food Centre,164027 Singapore,"[1.28532379322674, 103.845776735598]","[1.282908655, 103.824868]"
3,09005500003,1,09:00:55,09:45:55,60,60,Old Airport Road Food Centre,342106 Singapore,Old Airport Road Food Centre,342106 Singapore,"[1.30825157725793, 103.885833296453]","[1.333642111, 103.8711648]"
4,09005900004,1,09:00:59,09:45:59,60,60,Amoy Street Food Centre,102103 Singapore,Amoy Street Food Centre,102103 Singapore,"[1.2793398636571, 103.846652482254]","[1.280736924, 103.8131357]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1599,14570801599,1,14:57:08,15:42:08,60,60,Tiong Bahru Market,163127 Singapore,Tiong Bahru Market,163127 Singapore,"[1.28508829297302, 103.832348811484]","[1.281522804, 103.8300358]"
1600,14574601600,1,14:57:46,15:42:46,60,60,Alexandra Village Food Centre,120603 Singapore,Alexandra Village Food Centre,120603 Singapore,"[1.28629780818066, 103.804489336296]","[1.304788651, 103.7673293]"
1601,14575001601,1,14:57:50,15:42:50,60,60,Golden Mile Food Centre,338520 Singapore,Golden Mile Food Centre,338520 Singapore,"[1.30287553881161, 103.863883266692]","[1.321188763, 103.8650441]"
1602,14575601602,1,14:57:56,15:42:56,60,60,"66 Kg Bugis, Singapore 338987, Singapore","33 Jalan Nuri, Singapore 438456, Singapore","66 Kg Bugis, Singapore 338987, Singapore","33 Jalan Nuri, Singapore 438456, Singapore","[1.306059801, 103.8660694]","[1.299290754, 103.8938509]"


In [214]:
# write to file
instance.to_csv("Food_delivery_simulator/Benchmark/Instance_medium/Instance_1/instance_1.csv", index=False) # need update for different instance

### Batch generation

In [219]:
restaurant_list = ['Alexandra Village Food Centre',
                   'Bukit Merah View Market & Hawker Centre',
                   '417 River Valley Rd, Singapore 248316, Singapore',
                   'Zion Riverside Food Centre',
                   'Tiong Bahru Market',
                   '37 Cuppage Rd, Singapore 229460, Singapore',
                   '371 Beach Rd, Singapore 199597, Singapore',
                   'Golden Mile Food Centre',
                   '66 Kg Bugis, Singapore 338987, Singapore',
                   '36 Boat Quay, Singapore 049825, Singapore',
                   '54 South Bridge Rd, Singapore 058685, Singapore',
                   'Hong Lim Food Centre',
                   'Maxwell Food Centre',
                   'Amoy Street Food Centre',
                   'Old Airport Road Food Centre'
                  ]

In [220]:
order_number_list = []
for n in range(1500, 2000, 50):
    order_number_list.append(n)
print(order_number_list)

[1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, 1950]


In [221]:
driver_number_list_small, driver_number_list_large = [], []

for n in range(80, 120, 5):
    driver_number_list_small.append(n)
for n in range(100, 140, 5):
    driver_number_list_large.append(n)

driver_number_list_small, driver_number_list_large

([80, 85, 90, 95, 100, 105, 110, 115],
 [100, 105, 110, 115, 120, 125, 130, 135])

In [222]:
%%time
def batch_instance_generation(select_restaurant, batch_order_list, batch_driver_list_small, batch_driver_list_large):
    count = 1
    for order_number in batch_order_list:
        restaurant_order_number = order_num_generation(select_restaurant, order_number)
        instance_df = instance_generation(select_restaurant, restaurant_order_num=restaurant_order_number)
        
        # decide the number of drivers
        if order_number <= 1700:
            batch_driver_list = batch_driver_list_small
        else:
            batch_driver_list = batch_driver_list_large
        
        # write to file
        for driver_number in batch_driver_list:
            print(count)
            if not os.path.exists(f"Food_delivery_simulator/Benchmark/Instance_medium/Instance_{count}"): # path need be changed if other instances
                os.makedirs(f"Food_delivery_simulator/Benchmark/Instance_medium/Instance_{count}") # path need be changed if other instances
            
            # write instance file
            instance_df.to_csv(f"Food_delivery_simulator/Benchmark/Instance_medium/Instance_{count}/instance_{count}.csv", index=False) # path need be changed if other instances
            # write driver file
            driver_info_df = pd.DataFrame(columns=['car_num', 'capacity', 'operation_time', 'gps_id'])
            driver_info_df['car_num'] = ['D_'+str(i) for i in range(1, driver_number+1)]
            driver_info_df['capacity'] = 10
            driver_info_df['operation_time'] = 24
            driver_info_df['gps_id'] = ['G_'+str(i) for i in range(1, driver_number+1)]
            driver_info_df.to_csv(f"Food_delivery_simulator/Benchmark/Instance_medium/Instance_{count}/driver_info.csv", index=False) # path need be changed if other instances
            
            # update count
            count += 1
    return 

batch_instance_generation(restaurant_list, order_number_list, driver_number_list_small, driver_number_list_large)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
CPU times: user 16.2 s, sys: 593 ms, total: 16.8 s
Wall time: 37.8 s


## Route, customer, restaurant information

In [199]:
%%time
select_res_df = select_df.loc[select_df['pick_up_address'].isin(restaurant_list)]
select_res_df['delivery_address'] = select_res_df['delivery_address'].str.strip()
select_res_df['pick_up_address'] = select_res_df['pick_up_address'].str.strip()
select_res_df

CPU times: user 20.8 ms, sys: 84.6 ms, total: 105 ms
Wall time: 363 ms


/Users/yangjingfeng/anaconda3/envs/d2l/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yangjingfeng/anaconda3/envs/d2l/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,merchant_id,merchant_name,driver_id,driver_name,vehicle_type,order_id,pick_up_address,delivery_address,paid_fee,delivery_distance,...,picked_at,delivered_at,cancelled_at,driver_paid_fee,car_only,tips,Restaurant Area,pick_up_coordinate,delivery_coordinate,coordinate_distance
83,166,WhyQ Pte Ltd,990.0,Alvin Tan,0.0,WH0011,Amoy Street Food Centre,"31-33 Cairnhill Cir, Singapore 229775",7.7350,5.97 km,...,2021-03-16 13:10:58.708,2021-03-16 14:25:26.510,NaN,6.9850,False,0.0,NaN,"(1.2793398636571, 103.846652482254)","(1.308180497, 103.8387051)",3.326407
84,166,WhyQ Pte Ltd,990.0,Alvin Tan,0.0,WH0021,Maxwell Food Centre,Singapore,7.6200,4.66 km,...,2021-03-17 12:16:48.958,2021-03-17 12:49:53.428,NaN,6.8700,False,0.0,NaN,"(1.28033142727315, 103.844747227479)","(1.28791981662355, 103.830283766722)",1.815820
85,166,WhyQ Pte Ltd,676.0,Enver Neo,1.0,WH0022,Maxwell Food Centre,"114 Middle Rd, Singapore 188971",7.0200,3.86 km,...,2021-03-17 12:17:27.256,2021-03-17 13:07:45.183,NaN,6.2700,False,0.0,NaN,"(1.28033142727315, 103.844747227479)","(1.299169364, 103.8536286)",2.315707
86,166,WhyQ Pte Ltd,691.0,Ethan lim,3.0,WH0023,Maxwell Food Centre,"114 Kim Seng Rd, Singapore 239433",7.0125,3.85 km,...,2021-03-17 12:50:50.028,2021-03-17 13:26:21.369,NaN,6.2625,False,0.0,NaN,"(1.28033142727315, 103.844747227479)","(1.294063906, 103.8330446)",2.006026
87,166,WhyQ Pte Ltd,244.0,mageswaran rao,1.0,WH0024,Maxwell Food Centre,"32 Orange Grove Rd, Singapore 258354",8.7000,6.1 km,...,2021-03-17 12:18:14.406,2021-03-17 13:35:29.400,NaN,7.9500,False,0.0,NaN,"(1.28033142727315, 103.844747227479)","(1.313412137, 103.8257587)",4.241058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19424,421,Ru‚Äôs Kitchen ÂÑí‚Ä¢Âé®,8932.0,Prakash Valasamy,2.0,RU0E32F8,"371 Beach Rd, Singapore 199597, Singapore","530 Joo Chiat Rd, Singapore 427710, Singapore",13.3200,NaN,...,2021-09-11 18:22:02.343,2021-09-11 18:43:07.936,NaN,12.5700,False,0.0,NaN,"(1.302347543, 103.86226)","(1.304508831, 103.9054509)",4.807384
19437,166,WhyQ Pte Ltd,NaN,NaN,NaN,WHEF34A2,Alexandra Village Food Centre,141095 Singapore,15.2900,NaN,...,NaN,NaN,2021-09-11 18:46:25.541,13.7900,False,0.0,NaN,"(1.28630594274136, 103.804492637648)","(1.296297098, 103.8118996)",1.382841
19438,166,WhyQ Pte Ltd,NaN,NaN,NaN,WHEF34A2,Alexandra Village Food Centre,267952 Singapore,15.2900,NaN,...,NaN,NaN,2021-09-11 18:46:25.541,13.7900,False,0.0,NaN,"(1.28630594274136, 103.804492637648)","(1.314888998, 103.8034153)",3.180551
19440,353,Lucali BYGB,7865.0,Mark Tan,0.0,LU00F0E1,"66 Kg Bugis, Singapore 338987, Singapore","152K Tembeling Rd, Singapore 423480, Singapore",15.4300,NaN,...,NaN,NaN,NaN,14.6800,True,0.0,NaN,"(1.306059801, 103.8660694)","(1.309451797, 103.9028838)",4.109858


In [200]:
res_unique_pickup_id = select_res_df['pick_up_address'].unique()
res_unique_delivery_id = select_res_df['delivery_address'].unique()
res_unique_pickup_id.shape, res_unique_delivery_id.shape

((15,), (3037,))

In [201]:
import ast

# customer_coordinate_dict
def get_customer_coordinate(df, unique_delivery_id):
    customer_coordinate_dict = {}
    for q in unique_delivery_id:
        coordinate = df[df['delivery_address'] == q]['delivery_coordinate'].unique()[0]
        customer_coordinate_dict[q] = ast.literal_eval(coordinate)
    return customer_coordinate_dict

customer_coordinate_dict = get_customer_coordinate(select_res_df, res_unique_delivery_id)

In [202]:
customer_coordinate_dict

{'31-33 Cairnhill Cir, Singapore 229775': (1.308180497, 103.8387051),
 'Singapore': (1.28791981662355, 103.830283766722),
 '114 Middle Rd, Singapore 188971': (1.299169364, 103.8536286),
 '114 Kim Seng Rd, Singapore 239433': (1.294063906, 103.8330446),
 '32 Orange Grove Rd, Singapore 258354': (1.313412137, 103.8257587),
 '165 Tg Pagar Rd, Singapore 088539': (1.274973032, 103.8435952),
 '23 Marina Way, Singapore 018979': (1.277224256, 103.8536949),
 '22 Jalan Membina, Singapore 166022, Singapore': (1.284887947, 103.8264446),
 '21 Nassim Rd, Singapore 258462': (1.307888439, 103.8229907),
 '9 Bukit Pasoh Rd, Singapore 089823, Singapore': (1.278754367, 103.840654),
 '182 Cecil St, Singapore 069547': (1.279029214, 103.8474129),
 '77 Robinson Rd, Singapore 068896': (1.277770761, 103.8484496),
 '1091 Serangoon Rd, Singapore 328190, Singapore': (1.324233409, 103.864556),
 '21 Leonie Hl, Singapore 239223, Singapore': (1.29876715, 103.8317743),
 '6 Derbyshire Rd, Singapore 309462, Singapore': (1.

In [204]:
%%time
route_info_list = []
all_locations = np.append(res_unique_pickup_id, res_unique_delivery_id)
location_coordinate_dict = {**restaurant_coordinate_dict, **customer_coordinate_dict}

# distance matrix
for p in all_locations:
    for q in all_locations:
        L = [p, q]
        p_coordinate = location_coordinate_dict[p]
        q_coordinate = location_coordinate_dict[q]
        L.append(round(hs.haversine(p_coordinate, q_coordinate),2))
        route_info_list.append(L)

route_info = pd.DataFrame(route_info_list, columns=['start_location_id', 'end_location_id', 'distance'])
route_info['time'] = route_info['distance'] * 120 
route_info['time'] = route_info['time'].apply(np.int64)

route_info['route_code'] = str()
for index, row in route_info.iterrows():
    route_info.at[index, 'route_code'] = str(index).zfill(6)
first_col = route_info.pop('route_code')
route_info.insert(0, 'route_code', first_col)

CPU times: user 15min 29s, sys: 9.29 s, total: 15min 39s
Wall time: 16min 1s


In [205]:
route_info

,route_code,start_location_id,end_location_id,distance,time
0,000000,Amoy Street Food Centre,Amoy Street Food Centre,0.00,0
1,000001,Amoy Street Food Centre,Maxwell Food Centre,0.24,28
2,000002,Amoy Street Food Centre,"371 Beach Rd, Singapore 199597, Singapore",3.09,370
3,000003,Amoy Street Food Centre,"37 Cuppage Rd, Singapore 229460, Singapore",2.64,316
4,000004,Amoy Street Food Centre,"417 River Valley Rd, Singapore 248316, Singapore",2.58,309
...,...,...,...,...,...
9314699,9314699,"152K Tembeling Rd, Singapore 423480, Singapore",120610 Singapore,14.92,1790
9314700,9314700,"152K Tembeling Rd, Singapore 423480, Singapore",138686 Singapore,13.29,1594
9314701,9314701,"152K Tembeling Rd, Singapore 423480, Singapore",169483 Singapore,8.80,1056
9314702,9314702,"152K Tembeling Rd, Singapore 423480, Singapore",267952 Singapore,11.07,1328


In [206]:
# write to file
route_info.to_csv("Food_delivery_simulator/Benchmark/Instance_medium/route_info.csv", index=False)

### restaurant information

In [207]:
# restaurant
select_restaurant_coordinate_dict = {}
for r in restaurant_coordinate_dict:
    if r in restaurant_list:
        select_restaurant_coordinate_dict[r] = restaurant_coordinate_dict[r]

restaurant_info_df = pd.DataFrame(list(select_restaurant_coordinate_dict.items()), columns=['restaurant_id', 'coordinate'])

restaurant_info_df['latitude'] = 0.0
restaurant_info_df['longitude'] = 0.0

for index, row in restaurant_info_df.iterrows():
    restaurant_info_df.at[index, 'latitude'] = row['coordinate'][0]
    restaurant_info_df.at[index, 'longitude'] = row['coordinate'][1]

restaurant_info_df = restaurant_info_df[['restaurant_id', 'latitude', 'longitude']]
restaurant_info_df['dispatch_radius'] = 5
restaurant_info_df['customer_radius'] = 5
restaurant_info_df['wait_time'] = 120
restaurant_info_df

,restaurant_id,latitude,longitude,dispatch_radius,customer_radius,wait_time
0,Maxwell Food Centre,1.280330,103.844734,5,5,120
1,Hong Lim Food Centre,1.285324,103.845777,5,5,120
2,Old Airport Road Food Centre,1.308252,103.885833,5,5,120
3,"37 Cuppage Rd, Singapore 229460, Singapore",1.302226,103.840312,5,5,120
4,Alexandra Village Food Centre,1.286298,103.804489,5,5,120
5,Tiong Bahru Market,1.285088,103.832349,5,5,120
6,Bukit Merah View Market & Hawker Centre,1.285505,103.822049,5,5,120
7,Amoy Street Food Centre,1.279340,103.846652,5,5,120
8,Golden Mile Food Centre,1.302876,103.863883,5,5,120
9,"66 Kg Bugis, Singapore 338987, Singapore",1.306060,103.866069,5,5,120


In [208]:
# write to file
restaurant_info_df.to_csv("Food_delivery_simulator/Benchmark/Instance_medium/restaurant_info.csv", index=False) # need to update for different instance

### customer information

In [209]:
# customer
customer_info_df = pd.DataFrame(list(customer_coordinate_dict.items()), columns=['customer_id', 'coordinate'])

customer_info_df['latitude'] = 0.0
customer_info_df['longitude'] = 0.0

for index, row in customer_info_df.iterrows():
    customer_info_df.at[index, 'latitude'] = row['coordinate'][0]
    customer_info_df.at[index, 'longitude'] = row['coordinate'][1]

customer_info_df = customer_info_df[['customer_id', 'latitude', 'longitude']]
customer_info_df

,customer_id,latitude,longitude
0,"31-33 Cairnhill Cir, Singapore 229775",1.308180,103.838705
1,Singapore,1.287920,103.830284
2,"114 Middle Rd, Singapore 188971",1.299169,103.853629
3,"114 Kim Seng Rd, Singapore 239433",1.294064,103.833045
4,"32 Orange Grove Rd, Singapore 258354",1.313412,103.825759
...,...,...,...
3032,120610 Singapore,1.304547,103.768774
3033,138686 Singapore,1.306110,103.783360
3034,169483 Singapore,1.284513,103.827712
3035,267952 Singapore,1.314889,103.803415


In [210]:
# write to file
customer_info_df.to_csv("Food_delivery_simulator/Benchmark/Instance_medium/customer_info.csv", index=False) # need to update for different instance